In [2]:
import pandas as pd
df=pd.read_csv('consultants_list.csv')
df.shape

(7091, 3)

In [3]:
df2=pd.read_csv('new_consultants_list.csv')
df2.shape

(6520, 3)

In [4]:
import pandas as pd

# Load both CSV files
df = pd.read_csv('consultants_list.csv')         # All consultants (old and new)
df2 = pd.read_csv('new_consultants_list.csv')     # New consultants only

# Create a 'Full Name' column in df by combining 'First Name' and 'Last Name'
df['Full Name'] = df['First Name'] + ' ' + df['Last Name']

# Identify old consultants by excluding those in the new list
old_consultants = df[~df['Full Name'].isin(df2['Full Name'])]

# Display or save the old consultants list
print(old_consultants)
# Optionally, save to a new CSV
old_consultants.to_csv('old_consultants_list.csv', index=False)
old_consultants

           First Name      Last Name         Country              Full Name
30    Farid Alexander          Afnan  United Kingdom  Farid Alexander Afnan
33              Jamie          Agnew  United Kingdom            Jamie Agnew
51             Marina   Ajubita Diaz  United Kingdom    Marina Ajubita Diaz
76            Ibrahim      Alhindawi         Ireland      Ibrahim Alhindawi
83               Lucy          Allan  United Kingdom             Lucy Allan
...               ...            ...             ...                    ...
6888            Yifan          Zhang  United Kingdom            Yifan Zhang
7021          Stephen  Ziegler-Jones  United Kingdom  Stephen Ziegler-Jones
7031        Catherine          Zobel  United Kingdom        Catherine Zobel
7038          Stefano          Zucca  United Kingdom          Stefano Zucca
7041              May           Zune  United Kingdom               May Zune

[571 rows x 4 columns]


,First Name,Last Name,Country,Full Name
30,Farid Alexander,Afnan,United Kingdom,Farid Alexander Afnan
33,Jamie,Agnew,United Kingdom,Jamie Agnew
51,Marina,Ajubita Diaz,United Kingdom,Marina Ajubita Diaz
76,Ibrahim,Alhindawi,Ireland,Ibrahim Alhindawi
83,Lucy,Allan,United Kingdom,Lucy Allan
...,...,...,...,...
6888,Yifan,Zhang,United Kingdom,Yifan Zhang
7021,Stephen,Ziegler-Jones,United Kingdom,Stephen Ziegler-Jones
7031,Catherine,Zobel,United Kingdom,Catherine Zobel
7038,Stefano,Zucca,United Kingdom,Stefano Zucca


In [5]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart, Reference, PieChart

# Load datasets
old_consultants = pd.read_csv('old_consultants_list.csv')
new_consultants = pd.read_csv('new_consultants_list.csv')

# Calculate metrics
new_additions_count = len(new_consultants)
country_analysis = new_consultants['Country'].value_counts()
total_old = len(old_consultants)
total_new = total_old + new_additions_count
growth_rate = (new_additions_count / total_old) * 100 if total_old > 0 else None
top_contributors = country_analysis.head()
percentage_distribution = (country_analysis / new_additions_count) * 100

# Write data to Excel
wb = Workbook()
ws1 = wb.active
ws1.title = "Summary"

# Summary data
summary_data = [
    ["Metric", "Value"],
    ["Number of New Additions", new_additions_count],
    ["Total Consultants in Database", total_new],
    ["Growth Rate (%)", growth_rate]
]
for row in summary_data:
    ws1.append(row)

# Country Analysis sheet
ws2 = wb.create_sheet(title="Country Analysis")
for row in dataframe_to_rows(country_analysis.reset_index(name="New Additions"), index=False, header=True):
    ws2.append(row)

# Percentage Distribution sheet
ws3 = wb.create_sheet(title="Percentage Distribution")
for row in dataframe_to_rows(percentage_distribution.reset_index(name="Percentage"), index=False, header=True):
    ws3.append(row)

# Save the Excel file
wb.save("Consultant_Report.xlsx")


In [7]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.chart import BarChart, PieChart, Reference

# Load datasets
old_consultants = pd.read_csv('old_consultants_list.csv')
new_consultants = pd.read_csv('new_consultants_list.csv')

# Calculate metrics
new_additions_count = len(new_consultants)
country_analysis = new_consultants['Country'].value_counts()
total_old = len(old_consultants)
total_new = total_old + new_additions_count
growth_rate = (new_additions_count / total_old) * 100 if total_old > 0 else None
top_contributors = country_analysis.head(5)  # Limit to top 5 for simplicity
percentage_distribution = (country_analysis / new_additions_count) * 100

# Create a new workbook and add a summary sheet
wb = Workbook()
ws = wb.active
ws.title = "Consultant Report"

# Add summary metrics
ws.append(["Metric", "Value"])
ws.append(["New Additions", new_additions_count])
ws.append(["Total Consultants in Database", total_new])
ws.append(["Growth Rate (%)", growth_rate])

# Add a gap before the country analysis section
ws.append([""])
ws.append(["Country", "New Additions", "Percentage"])

# Add country analysis data
for country, count in top_contributors.items():
    ws.append([country, count, percentage_distribution[country]])

# Adding Bar Chart for Country Analysis
bar_chart = BarChart()
bar_chart.title = "Top 5 Countries by New Additions"
bar_chart.x_axis.title = "Country"
bar_chart.y_axis.title = "New Additions"
data = Reference(ws, min_col=2, min_row=6, max_row=10)
categories = Reference(ws, min_col=1, min_row=7, max_row=10)
bar_chart.add_data(data, titles_from_data=True)
bar_chart.set_categories(categories)
bar_chart.height = 7  # Adjust chart size
bar_chart.width = 10
ws.add_chart(bar_chart, "E5")  # Place chart on the right side of the summary data

# Adding Pie Chart for Percentage Distribution
pie_chart = PieChart()
pie_chart.title = "Percentage Distribution by Country"
data = Reference(ws, min_col=3, min_row=6, max_row=10)
labels = Reference(ws, min_col=1, min_row=7, max_row=10)
pie_chart.add_data(data, titles_from_data=True)
pie_chart.set_categories(labels)
pie_chart.height = 7  # Adjust chart size
pie_chart.width = 10
ws.add_chart(pie_chart, "E20")  # Place below the bar chart

# Save the Excel report
wb.save("Consultant_Summary_Report_with_Charts.xlsx")
